In [14]:
import numpy as np
import pandas as pd
import time
import datetime
import sys
import os
from tqdm import tqdm
from functools import reduce
from Bio import SeqIO
import joblib
sys.path.append("../tools/")
import funclib

sys.path.append("../")
import benchmark_train as btrain
import benchmark_test as btest
import benchmark_common as bcommon
import config as cfg
import benchmark_evaluation as eva
import embedding_esm as esmebd

import production as pdc

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from pandarallel import pandarallel #  import pandaralle
pandarallel.initialize() # init

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
#read train test data
train = pd.read_feather(cfg.DATADIR+'task2/train.feather')
test = pd.read_feather(cfg.DATADIR+'task2/test.feather')
print('train size: {0}\ntest size: {1}'.format(len(train), len(test)))

train size: 222567
test size: 3304


In [16]:
train_unirep = pd.read_feather(cfg.DATADIR + 'train_unirep.feather')
test_unirep = pd.read_feather(cfg.DATADIR + 'test_unirep.feather')

In [18]:
trainset = train.copy()
testset = test.copy()

train_unirep = trainset.merge(train_unirep, on='id', how='left')
test_unirep = testset.merge(test_unirep, on='id', how='left')

In [20]:
X_train =np.array(train_unirep.iloc[:,3:])
X_test = np.array(test_unirep.iloc[:,3:])

Y_train = np.array(train_unirep.functionCounts.astype('int')-1).flatten()
Y_test = np.array(test_unirep.functionCounts.astype('int')-1).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test, type='multi')

 baslineName 		 accuracy 	 precision-macro 	 recall-macro 	 f1-macro
         knn  		0.904358  	0.578957 		0.147934 	0.147384
          lr  		0.923426  	0.846244 		0.142810 	0.137170
          xg  		0.919189  	0.882248 		0.148026 	0.147480
          dt  		0.864709  	0.595059 		0.142983 	0.143993
          rf  		0.921005  	0.846217 		0.142436 	0.136983
        gbdt  		0.922821  	0.846237 		0.142717 	0.137123
